In [1]:
## import pandas as pd
import numpy as np
import re

In [2]:
data_dir = "original data/"
output_dir = "output/"

In [3]:
with open(data_dir + 'wiki_q_train.txt', 'r') as f:
    train = f.read()
with open(data_dir + 'wiki_q_valid.txt', 'r') as f:
    valid = f.read()
with open(data_dir + 'wiki_q_test.txt', 'r') as f:
    test = f.read()

In [4]:
train = train.split("\n")
train = [line.split("\t") for line in train]

In [5]:
valid = valid.split("\n")
valid = [line.split("\t") for line in valid]

In [6]:
test = test.split("\n")
test = [line.split("\t") for line in test]

In [7]:
len(test)

63111

In [8]:
whole = train + valid + test

In [9]:
len(whole)

669937

In [11]:
entity_set = set()
relation_set = set()
for line in whole:
    p = re.compile(r'^P\d')
    q = re.compile(r'^Q\d')
    p_list = [ s for s in line if p.match(s) ]
    q_list = [ s for s in line if q.match(s) ]
    for word in p_list:
        relation_set.add(word)
    for word in q_list:
        entity_set.add(word)
    for idx, word in enumerate(line):
        if len(word)>10 and idx <=2:
            print(line)

In [12]:
len(relation_set)

127

In [13]:
len(entity_set)

19213

In [46]:
whole_refined = []
len_dict = dict()
total_num = 0
original_num = 0
for line in whole:
    if line != ['']:
        tmp = line[:4]
        tmp2 = line[4:]
        qualifier = []
        count = 0
        for idx, word in enumerate(tmp2):
            if idx%2 ==0:
                original_num += 1
                if (p.match(word) and q.match(tmp2[idx+1])) or (word in ["P580", "P582"] and len(tmp2[idx+1]) == 20):
                
#                 if p.match(word) or q.match(word):
#                     if p.match(tmp2[idx+1]) or q.match(tmp2[idx+1]):
                    qualifier.append(word)
                    qualifier.append(tmp2[idx+1])
                    count += 1
        whole_refined.append(tmp + qualifier)
        len_dict[count] = len_dict.get(count, 0) + 1
        total_num +=1
    

In [47]:
len('1832-01-01T00:00:00Z')

20

In [48]:
whole_refined

[['Q838242', 'P31', 'Q484170', '0', 'P582', '2015-12-31T00:00:00Z'],
 ['Q232308',
  'P69',
  'Q499451',
  '0',
  'P812',
  'Q11451',
  'P582',
  '1915-01-01T00:00:00Z',
  'P512',
  'Q787674'],
 ['Q683522', 'P27', 'Q161885', '0', 'P582', '1800-12-31T00:00:00Z'],
 ['Q663864', 'P31', 'Q484170', '0', 'P582', '2016-12-31T00:00:00Z'],
 ['Q212429', 'P150', 'Q16987', '0', 'P582', '2016-01-01T00:00:00Z'],
 ['Q158584',
  'P17',
  'Q153136',
  '0',
  'P582',
  '1804-08-10T00:00:00Z',
  'P580',
  '1526-01-01T00:00:00Z'],
 ['Q42457', 'P131', 'Q15121', '0', 'P582', '2014-12-31T00:00:00Z'],
 ['Q72265', 'P131', 'Q15123', '0', 'P582', '2014-12-31T00:00:00Z'],
 ['Q15336',
  'P17',
  'Q34266',
  '0',
  'P580',
  '1721-01-01T00:00:00Z',
  'P582',
  '1917-03-15T00:00:00Z'],
 ['Q12589', 'P131', 'Q16954', '0', 'P582', '2015-12-31T00:00:00Z'],
 ['Q540925',
  'P69',
  'Q675848',
  '0',
  'P582',
  '1974-01-01T00:00:00Z',
  'P812',
  'Q208217',
  'P512',
  'Q752297'],
 ['Q496989', 'P131', 'Q16227', '0', 'P582',

In [49]:
total_num

669934

In [50]:
original_num

1312429

In [51]:
entity_set = set()
relation_set = set()
for line in whole_refined:
    p = re.compile(r'^P\d')
    q = re.compile(r'^Q\d')
    p_list = [ s for s in line if p.match(s) ]
    q_list = [ s for s in line if q.match(s) ]
    for word in p_list:
        relation_set.add(word)
    for word in q_list:
        entity_set.add(word)
    for idx, word in enumerate(line):
        if len(word)>10 and idx <=2:
            print(line)

In [52]:
entity2idx = dict()
with open(output_dir + "entity2id.txt", "w") as f:
    for idx, i in enumerate(entity_set):
        f.write(i + "\t" + str(idx))
        entity2idx[i] = str(idx)
        if idx != len(entity_set)-1:
            f.write("\n")

In [53]:
relation2idx = dict()
with open(output_dir + "relation2id.txt", "w") as f:
    for idx, i in enumerate(relation_set):
        f.write(i + "\t" + str(idx))
        relation2idx[i] = str(idx)
        if idx != len(relation_set)-1:
            f.write("\n")
        

In [54]:
len(relation2idx)

97

In [55]:
train = open(output_dir + "train.txt", 'w')
valid = open(output_dir + "valid.txt", 'w')
test = open(output_dir + "test.txt", 'w')

with open(output_dir + "data.txt", 'w') as f:
    for line_idx, line in enumerate(whole_refined):
        out = ''
        for idx, word in enumerate(line):
            if idx != 3:
                if word[0] == "P":
                    out += relation2idx[word]
                elif word[0] == "Q":
                    out += entity2idx[word]
                else:
                    out += word
            else:
                out += word
            out += "\t"
            
        if line_idx != len(whole_refined)-1:
            out = out[:-1]
            out += "\n"
        else:
            out = out[:-1]
        f.write(out)
        
        if int(line[3]) >= 211 and int(line[3]) < 222:
            valid.write(out)
        elif int(line[3]) >= 222:
            test.write(out)
        else:
            train.write(out)
            
train.close()
valid.close()
test.close()
                    

In [56]:
len_dict

{1: 338745,
 3: 71195,
 2: 157564,
 5: 8609,
 0: 58130,
 4: 21062,
 8: 1796,
 6: 3664,
 7: 4256,
 40: 227,
 15: 575,
 9: 489,
 12: 498,
 11: 240,
 21: 59,
 14: 150,
 10: 445,
 16: 140,
 22: 59,
 24: 65,
 13: 201,
 41: 54,
 19: 169,
 35: 101,
 17: 168,
 32: 70,
 28: 58,
 27: 108,
 29: 77,
 30: 24,
 34: 139,
 18: 75,
 47: 36,
 90: 52,
 23: 202,
 33: 48,
 37: 19,
 26: 33,
 20: 161,
 56: 29,
 63: 10,
 25: 58,
 48: 21,
 53: 7,
 54: 27,
 36: 19}

In [57]:
k=["P150", "Q12903", "Power"]

In [58]:
l2 = [ s for s in k if p.match(s) ]

In [71]:
l2

['P150']

In [59]:
f = open('output/entity2id.txt', 'r')

In [60]:
data = f.read()
data = data.split("\n")
data = [i.split("\t") for i in data]

# Description

In [61]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

with open('output/entity2id_description.txt', 'w') as  f:
    for qnode, idx in tqdm(data):
        connected = False    
        while connected == False:
            try:
                query = '''
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX schema: <http://schema.org/>

                SELECT ?o
                WHERE 
                {
                  wd:''' + qnode + ''' schema:description ?o.
                  FILTER ( lang(?o) = "en" )
                }
                '''
                sparql.setQuery(query)
                sparql.setReturnFormat(JSON)

                results = sparql.query().convert()
                try:
                    res = results["results"]['bindings'][0]['o']['value']
        #             print(qnode + "\t" + res)

                    f.write(qnode + "\t" + idx + "\t" + res + "\n")
                except:
                    f.write(qnode + "\t" + idx + "\n")
                connected = True
            except:
                print("error")

    
    
    

  0%|▌                                                                                                                                                     | 65/19213 [00:22<1:26:13,  3.70it/s]

error


  0%|▌                                                                                                                                                     | 66/19213 [00:24<2:52:48,  1.85it/s]

error


  0%|▌                                                                                                                                                     | 73/19213 [00:26<1:39:21,  3.21it/s]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 19085/19213 [2:04:30<00:38,  3.34it/s]

error


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 19086/19213 [2:04:31<00:45,  2.81it/s]

error


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19213/19213 [2:05:17<00:00,  2.56it/s]


In [62]:
print(query)


                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX schema: <http://schema.org/>

                SELECT ?o
                WHERE 
                {
                  wd:Q347423 schema:description ?o.
                  FILTER ( lang(?o) = "en" )
                }
                


In [63]:
f = open('output/relation2id.txt', 'r')

In [64]:
data = f.read()
data = data.split("\n")
data = [i.split("\t") for i in data]

In [65]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

with open('output/relation2id_description.txt', 'w') as  f:
    for qnode, idx in tqdm(data):
        query = '''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX schema: <http://schema.org/>

        SELECT ?o
        WHERE 
        {
          wd:''' + qnode + ''' schema:description ?o.
          FILTER ( lang(?o) = "en" )
        }
        '''
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        results = sparql.query().convert()
        try:
            res = results["results"]['bindings'][0]['o']['value']
#             print(qnode + "\t" + res)

            f.write(qnode + "\t" + idx + "\t" + res + "\n")
        except:
            f.write(qnode + "\t" + idx + "\n")
    
    
    
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:33<00:00,  2.93it/s]


In [66]:
qnode

'P131'

In [40]:
query = '''
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>

SELECT ?o
WHERE 
{
  wd:''' + qnode + ''' schema:description ?o.
  FILTER ( lang(?o) = "en" )
}
'''

In [41]:
print(query)


PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>

SELECT ?o
WHERE 
{
  wd:P131 schema:description ?o.
  FILTER ( lang(?o) = "en" )
}

